In [18]:
# Load the Drive helper and mount
from google.colab import drive

# This will proot for authorization
drive.mount('/content/drive')

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import pickle
from collections import Counter
import io

lemmatizer = WordNetLemmatizer()
hm_lines = 10000000

def create_lexicon(pos, neg):
  lexicon = []
  for fi in [pos, neg]:
    with io.open(fi, 'r', encoding='cp437') as f:
      contents = f.readlines()
      for l in contents[:hm_lines]:
        all_words = word_tokenize(l.lower())
        lexicon += list(all_words)
  
  lexicon = [lemmatizer.lemmatize(i) for i in lexicon]
  w_counts = Counter(lexicon)
  l2 = []
  for w in w_counts:
    if 1000 > w_counts[w] > 50:
      l2.append(w)
  print(len(l2))
  return l2
  
def sample_handling(sample, lexicon, classification):
  featureset = []
  
  with io.open(sample, 'r', encoding='cp437') as f:
    contents = f.readlines()
    for l in contents[:hm_lines]:
      current_words = word_tokenize(l.lower())
      current_words = [lemmatizer.lemmatize(i) for i in current_words]
      features = np.zeros(len(lexicon))
      for word in current_words:
        if word.lower() in lexicon:
          index_value = lexicon.index(word.lower())
          features[index_value] += 1
      features = list(features)
      featureset.append([features, classification])
      
  return featureset

def create_feature_sets_and_labels(pos, neg, test_size=0.1):
  lexicon = create_lexicon(pos, neg)
  features = []
  pos_path = '/content/drive/My Drive/Colab Notebooks/ProcessingOurOwnData/pos.txt'
  neg_path = '/content/drive/My Drive/Colab Notebooks/ProcessingOurOwnData/neg.txt'
  features += sample_handling(pos_path, lexicon, [1, 0])
  features += sample_handling(neg_path, lexicon, [0, 1])
  random.shuffle(features)
  
  '''
  does tf.argmax([output]) == tf.argmax([expectations])
  tf.argmax([4234, 7923]) == tf.argmax([1, 0])
  # neural net will try to shift weights to make statement true
  '''
  features = np.array(features)
  testing_size = int(test_size*len(features))
  
  train_x = list(features[:,0][:-testing_size])
  train_y = list(features[:,1][:-testing_size])
  
  test_x = list(features[:,0][-testing_size:])
  test_y = list(features[:,1][-testing_size:])
  
  return train_x, train_y, test_x, test_y

if __name__ == '__main__':
  pos_path = '/content/drive/My Drive/Colab Notebooks/ProcessingOurOwnData/pos.txt'
  neg_path = '/content/drive/My Drive/Colab Notebooks/ProcessingOurOwnData/neg.txt'
  train_x, train_y, test_x, test_y = create_feature_sets_and_labels(pos_path, neg_path)
  with open('/content/drive/My Drive/Colab Notebooks/ProcessingOurOwnData/sentiment_set.pickle', 'wb') as f:
    pickle.dump([train_x, train_y, test_x, test_y], f)
  

Mounted at /content/drive
423


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will proot for authorization
drive.mount('/content/drive')

# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/Colab Notebooks/ProcessingOurOwnData/"